In [1]:
# =========================================================
# 04_global_network_map.ipynb
# Electrum Observatory — Global Network Map
# =========================================================

import json
import pandas as pd
import numpy as np
import requests
from pathlib import Path
import plotly.express as px
import time

DATA_DIR = Path("../../data")


# =========================================================
# 1. Load all available peer data
# =========================================================

def load_json(path):
    with open(path, "r") as f:
        return json.load(f)

peers_raw = load_json(DATA_DIR / "peers" / "peers.json")
online_raw = load_json(DATA_DIR / "online_peers" / "online_peers.json")

print("Total peers scanned:", len(peers_raw))
print("Peers seen online:", len(online_raw))

Total peers scanned: 3828
Peers seen online: 1665


In [2]:
# =========================================================
# 2. Convert JSON to DataFrames
# =========================================================

df_peers = pd.DataFrame(peers_raw)
df_online = pd.DataFrame(online_raw)

# Normalize host column (sometimes nested)
if "host" not in df_peers.columns:
    df_peers["host"] = df_peers["raw"].apply(lambda x: x[0] if isinstance(x, list) else None)

if "host" not in df_online.columns:
    df_online["host"] = df_online["raw"].apply(lambda x: x[0] if isinstance(x, list) else None)

# Basic cleanup
df_peers = df_peers.drop_duplicates(subset=["host"])
df_online = df_online.drop_duplicates(subset=["host"])

# Join online metadata into peers
df = df_peers.merge(df_online, on="host", how="left", suffixes=("", "_online"))

print("Unified dataframe:", df.shape)
df.head()

Unified dataframe: (291, 9)


,host,ssl,tcp,raw,port,protocol,latency_ms,version_raw,banner_raw
0,137.184.244.174,NaN,NaN,"[137.184.244.174, 137.184.244.174, [v1.4.3, s5...",50001.0,tcp,5856.29,"{""jsonrpc"":""2.0"",""result"":[""ElectrumX 1.18.0"",...","{""jsonrpc"":""2.0"",""result"":""You are connected t..."
1,194.233.69.180,NaN,50001.0,"[194.233.69.180, electrumx.dev, [v1.4.3, s5000...",50001.0,tcp,6076.51,"{""jsonrpc"":""2.0"",""result"":[""ElectrumX 1.18.0"",...","{""jsonrpc"":""2.0"",""result"":""You are connected t..."
2,37.27.18.174,NaN,NaN,"[37.27.18.174, 37.27.18.174, [v1.4.3, s50002]]",NaN,NaN,NaN,NaN,NaN
3,18.221.79.132,NaN,50001.0,"[18.221.79.132, btc5.publicrypto.com, [v1.4.3,...",NaN,NaN,NaN,NaN,NaN
4,18.217.9.91,50002.0,50001.0,"[18.217.9.91, btc.byte-share.com, [v1.4.3, s60...",NaN,NaN,NaN,NaN,NaN


In [3]:
# =========================================================
# 3. Prepare list of unique hosts for GeoIP lookup
# =========================================================

hosts = df["host"].dropna().unique().tolist()
print("Unique IPs:", len(hosts))

# Cache file to avoid redoing lookups
CACHE_FILE = Path("geo_cache.json")

if CACHE_FILE.exists():
    geo_cache = json.load(open(CACHE_FILE))
else:
    geo_cache = {}

Unique IPs: 291


In [4]:
# =========================================================
# 4. Batch GeoIP Lookup (ip-api.com/batch)
# =========================================================

def geo_batch_lookup(host_list):
    """
    Performs batch lookup of up to 100 IPs per request.
    ip-api.com/batch supports 45 requests per minute.
    """
    results = []
    batch_size = 100

    for i in range(0, len(host_list), batch_size):
        batch = host_list[i:i + batch_size]

        # Filter out cached
        to_query = [ip for ip in batch if ip not in geo_cache]

        if not to_query:
            # all cached
            results.extend([geo_cache[ip] for ip in batch])
            continue

        payload = [{"query": ip} for ip in to_query]

        r = requests.post("http://ip-api.com/batch", json=payload, timeout=10)
        data = r.json()

        # Save into cache
        for entry, ip in zip(data, to_query):
            geo_cache[ip] = entry

        results.extend([geo_cache[ip] for ip in batch])

        time.sleep(1.2)  # rate limiting

    # persist cache
    json.dump(geo_cache, open(CACHE_FILE, "w"), indent=2)

    return results


print("Running batch GeoIP lookup...")
geo_data = geo_batch_lookup(hosts)

geo_df = pd.DataFrame(geo_data)
geo_df.head()

Running batch GeoIP lookup...


,status,country,countryCode,region,regionName,city,zip,lat,lon,timezone,isp,org,as,query,message
0,success,United States,US,CA,California,Santa Clara,95054,37.3931,-121.9620,America/Los_Angeles,"DigitalOcean, LLC",Digital Ocean,"AS14061 DigitalOcean, LLC",137.184.244.174,NaN
1,success,Singapore,SG,01,Central Singapore,Singapore,,1.2821,103.8510,Asia/Singapore,Contabo Asia Private Limited,Contabo Asia Private Limited,AS141995 Contabo Asia Private Limited,194.233.69.180,NaN
2,success,Finland,FI,18,Uusimaa,Helsinki,00201,60.1719,24.9347,Europe/Helsinki,Hetzner Online GmbH,Hetzner Online GmbH,AS24940 Hetzner Online GmbH,37.27.18.174,NaN
3,success,United States,US,OH,Ohio,Dublin,43017,40.0992,-83.1141,America/New_York,"Amazon.com, Inc.",AWS EC2 (us-east-2),"AS16509 Amazon.com, Inc.",18.221.79.132,NaN
4,success,United States,US,OH,Ohio,Dublin,43017,40.0992,-83.1141,America/New_York,"Amazon.com, Inc.",AWS EC2 (us-east-2),"AS16509 Amazon.com, Inc.",18.217.9.91,NaN


In [5]:
# =========================================================
# 5. Merge GeoIP data into main dataframe
# =========================================================

df_geo = geo_df.rename(columns={
    "query": "host",
    "lat": "lat",
    "lon": "lon",
    "country": "country",
    "as": "asn"
})

df = df.merge(df_geo[["host", "lat", "lon", "country", "asn"]], on="host", how="left")

print(df[["host", "lat", "lon", "country"]].head())

              host      lat       lon        country
0  137.184.244.174  37.3931 -121.9620  United States
1   194.233.69.180   1.2821  103.8510      Singapore
2     37.27.18.174  60.1719   24.9347        Finland
3    18.221.79.132  40.0992  -83.1141  United States
4      18.217.9.91  40.0992  -83.1141  United States


In [6]:
# =========================================================
# 6. Clean invalid geodata
# =========================================================

df = df.dropna(subset=["lat", "lon"])
df = df[df["lat"] != 0]
df = df[df["lon"] != 0]

print("Valid geolocated servers:", len(df))
df.head()

Valid geolocated servers: 254


,host,ssl,tcp,raw,port,protocol,latency_ms,version_raw,banner_raw,lat,lon,country,asn
0,137.184.244.174,NaN,NaN,"[137.184.244.174, 137.184.244.174, [v1.4.3, s5...",50001.0,tcp,5856.29,"{""jsonrpc"":""2.0"",""result"":[""ElectrumX 1.18.0"",...","{""jsonrpc"":""2.0"",""result"":""You are connected t...",37.3931,-121.9620,United States,"AS14061 DigitalOcean, LLC"
1,194.233.69.180,NaN,50001.0,"[194.233.69.180, electrumx.dev, [v1.4.3, s5000...",50001.0,tcp,6076.51,"{""jsonrpc"":""2.0"",""result"":[""ElectrumX 1.18.0"",...","{""jsonrpc"":""2.0"",""result"":""You are connected t...",1.2821,103.8510,Singapore,AS141995 Contabo Asia Private Limited
2,37.27.18.174,NaN,NaN,"[37.27.18.174, 37.27.18.174, [v1.4.3, s50002]]",NaN,NaN,NaN,NaN,NaN,60.1719,24.9347,Finland,AS24940 Hetzner Online GmbH
3,18.221.79.132,NaN,50001.0,"[18.221.79.132, btc5.publicrypto.com, [v1.4.3,...",NaN,NaN,NaN,NaN,NaN,40.0992,-83.1141,United States,"AS16509 Amazon.com, Inc."
4,18.217.9.91,50002.0,50001.0,"[18.217.9.91, btc.byte-share.com, [v1.4.3, s60...",NaN,NaN,NaN,NaN,NaN,40.0992,-83.1141,United States,"AS16509 Amazon.com, Inc."


In [7]:
# =========================================================
# 7. Global Electrum Network Map (Plotly)
# =========================================================

fig = px.scatter_geo(
    df,
    lat="lat",
    lon="lon",
    hover_name="host",
    hover_data=["country", "asn"],
    color="country",
    projection="natural earth",
    title="Global Electrum Network Map",
    opacity=0.8,
)

fig.update_layout(
    height=650,
    margin=dict(l=0, r=0, t=40, b=0),
    geo=dict(
        bgcolor="rgba(0,0,0,0)",
        landcolor="rgb(30,30,30)",
        oceancolor="rgb(10,10,20)",
        showocean=True,
        projection_scale=1.05,
    )
)

fig.show()

In [8]:
# =========================================================
# 8. (Optional) Export the figure to PNG/SVG for the website
# =========================================================

fig.write_image("global_network_map.png", scale=3)
fig.write_image("global_network_map.svg", scale=3)

print("Exported global_network_map.png and .svg")

Exported global_network_map.png and .svg
